# Federated Protein Embeddings and Task Model Fitting with BioNeMo

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This notebook will walk you through the task fitting workflow in the following sections:

* 
*
*

### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, users can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein my be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [ ]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [ ]:
import io
import requests
from Bio import SeqIO

# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

### Data splitting

### Federated embedding extraction

In [20]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/embeddings",
    workspace="/tmp/nvflare/bionemo/embeddings",
    n_clients=1,
    threads=1
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-07-26 23:31:47,545 - SimulatorRunner - INFO - Create the Simulator Server.
2023-07-26 23:31:47,549 - Cell - INFO - server: creating listener on tcp://0:54173
2023-07-26 23:31:47,568 - Cell - INFO - server: created backbone external listener for tcp://0:54173
2023-07-26 23:31:47,569 - ConnectorManager - INFO - 11287: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-07-26 23:31:47,571 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:49596] is starting
2023-07-26 23:31:48,073 - Cell - INFO - server: created backbone internal listener for tcp://localhost:49596
2023-07-26 23:31:48,077 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:54173] is starting
2023-07-26 23:31:48,167 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 56345
2023-07-26 23:31:48,169 - SimulatorRunner - INFO - Deploy the Apps.
2023-07-26 23:31:48,698 - SimulatorRunner - INFO - Create the simulat

[NeMo W 2023-07-26 23:32:03 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-07-26 23:32:04 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-07-26 23:32:04,629 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=ea155ca7-93ac-491c-9edf-c9d81ce62685]: assigned task to client site-1: name=bionemo_inference, id=ea155ca7-93ac-491c-9edf-c9d81ce62685
2023-07-26 23:32:04,634 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=ea155ca7-93ac-491c-9edf-c9d81ce62685]: sent task assignment to client. client_name:site-1 task_id:ea155ca7-93ac-491c-9edf-c9d81ce62685
2023-07-26 23:32:04,636 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: bionemo_inference   task_id: ea155ca7-93ac-491c-9edf-c9d81ce62685  sharable_header_task_id: ea155ca7-93ac-491c-9edf-c9d81ce62685
[NeMo I 2023-07-26 23:32:04 utils:250] Restoring model from /tmp/nvflare/bionemo/embeddings/simulate_job/app_site-1/m

[NeMo W 2023-07-26 23:32:04 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/pytorch_lightning/plugins/precision/native_amp.py:131: LightningDeprecationWarning: The `NativeMixedPrecisionPlugin` class has been renamed in v1.9.0 and will be removed in v2.0.0. Please use `pytorch_lightning.plugins.MixedPrecisionPlugin` instead.
      rank_zero_deprecation(
    
I0726 23:32:04.770546 140367538272064 setup.py:163] GPU available: True (cuda), used: True
I0726 23:32:04.770993 140367538272064 setup.py:166] TPU available: False, using: 0 TPU cores
I0726 23:32:04.771120 140367538272064 setup.py:169] IPU available: False, using: 0 IPUs
I0726 23:32:04.771204 140367538272064 setup.py:172] HPU available: False, using: 0 HPUs
[NeMo E 2023-07-26 23:32:04 exp_manager:306] exp_manager did not receive a cfg argument. It will be disabled.


[NeMo I 2023-07-26 23:32:05 megatron_init:231] Rank 0 has data parallel group: [0]
[NeMo I 2023-07-26 23:32:05 megatron_init:234] All data parallel group ranks: [[0]]
[NeMo I 2023-07-26 23:32:05 megatron_init:235] Ranks 0 has data parallel rank: 0
[NeMo I 2023-07-26 23:32:05 megatron_init:243] Rank 0 has model parallel group: [0]
[NeMo I 2023-07-26 23:32:05 megatron_init:244] All model parallel group ranks: [[0]]
[NeMo I 2023-07-26 23:32:05 megatron_init:254] Rank 0 has tensor model parallel group: [0]
[NeMo I 2023-07-26 23:32:05 megatron_init:258] All tensor model parallel group ranks: [[0]]
[NeMo I 2023-07-26 23:32:05 megatron_init:259] Rank 0 has tensor model parallel rank: 0
[NeMo I 2023-07-26 23:32:05 megatron_init:273] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2023-07-26 23:32:05 megatron_init:285] Rank 0 has embedding group: [0]
[NeMo I 2023-07-26 23:32:05 megatron_init:291] All pipeline model parallel group ranks: [[0]]
[NeMo I 2023-07-26 23:32:05 megatron_init:292]

[NeMo W 2023-07-26 23:32:05 modelPT:245] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
I0726 23:32:05.220477 140367538272064 distributed.py:244] Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
I0726 23:32:05.221726 140367538272064 distributed.py:248] ----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

[NeMo W 2023-07-26 23:32:05 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/apex/transformer/pipeline_parallel/utils.py:81: UserWarning: This function is only for unittest
      warnings.warn("This function is only for unittest")
    


[NeMo I 2023-07-26 23:32:05 text_memmap_dataset:349] Time building 0 / 1 mem-mapped files: 0:00:00.130736
[NeMo I 2023-07-26 23:32:05 text_memmap_dataset:114] Loading data files
[NeMo I 2023-07-26 23:32:05 text_memmap_dataset:205] Loading /tmp/data/FLIP/secondary_structure/test/x000.csv
[NeMo I 2023-07-26 23:32:05 text_memmap_dataset:117] Time loading 1 mem-mapped files: 0:00:00.002499
[NeMo I 2023-07-26 23:32:05 text_memmap_dataset:121] Computing global indices
[NeMo I 2023-07-26 23:32:06 mapped_dataset:206] Filtered out (ignored) 24 samples ( 340 / 364 )
2023-07-26 23:32:06,218 - pytorch_lightning.accelerators.cuda - INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Predicting: 0it [00:00, ?it/s]

I0726 23:32:06.218470 140367538272064 cuda.py:58] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 3/3 [00:04<00:00,  1.39s/it]2023-07-26 23:32:11,629 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job]: got result from client site-1 for task: name=bionemo_inference, id=ea155ca7-93ac-491c-9edf-c9d81ce62685
2023-07-26 23:32:11,633 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, peer_rc=OK, task_name=bionemo_inference, task_id=ea155ca7-93ac-491c-9edf-c9d81ce62685]: finished processing client result by bionemo_inference
2023-07-26 23:32:11,635 - SubmitUpdateCommand - INFO - submit_update process. client_name:site-1   task_id:ea155ca7-93ac-491c-9edf-c9d81ce62685
2023-07-26 23:32:11,787 - BioNeMoInference - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference]: task bionemo_inference exit with status TaskCompletionStatus.OK
2023-07-26 23:32:11,804 - BioNeMoModelSharer - IN